Practical 1: Sentiment Detection in Movie Reviews
========================================



This practical concerns detecting sentiment in movie reviews. This is a typical NLP classification task.
In [this file](https://gist.githubusercontent.com/bastings/d47423301cca214e3930061a5a75e177/raw/5113687382919e22b1f09ce71a8fecd1687a5760/reviews.json) (80MB) you will find 1000 positive and 1000 negative **movie reviews**.
Each review is a **document** and consists of one or more sentences.

To prepare yourself for this practical, you should
have a look at a few of these texts to understand the difficulties of
the task: how might one go about classifying the texts? You will write
code that decides whether a movie review conveys positive or
negative sentiment.

Please make sure you have read the following paper:

>   Bo Pang, Lillian Lee, and Shivakumar Vaithyanathan
(2002).
[Thumbs up? Sentiment Classification using Machine Learning
Techniques](https://dl.acm.org/citation.cfm?id=1118704). EMNLP.

Bo Pang et al. were the "inventors" of the movie review sentiment
classification task, and the above paper was one of the first papers on
the topic. The first version of your sentiment classifier will do
something similar to Pang et al.'s system. If you have questions about it,
you should resolve as soon as possible with your TA.


**Advice**

Please read through the entire practical and familiarise
yourself with all requirements before you start coding or otherwise
solving the tasks. Writing clean and concise code can make the difference
between solving the assignment in a matter of hours, and taking days to
run all experiments.

**Implementation**

While we inserted code cells to indicate where you should implement your own code, please feel free to add/remove code blocks where you see fit (but make sure that the general structure of the assignment is preserved). Also, please keep in mind that it is always good practice to structure your code properly, e.g., by implementing separate classes and functions that can be reused.

Generally, any function that we already use or import in the notebook can be used. Besides those, importing something like `deepcopy` or `tqdm` that does not change the implementation of the algorithms is fine. Functions that change or simplify the way you would implement the algorithm, including text processing functions from libraries like `sklearn`, `pandas` or `nltk` are not allowed unless specified (e.g. `ngrams` and `PorterStemmer` from `ntlk`). If you have questions about any specific function or library, please discuss this with your TA who will be the one grading your assignment.

## Environment

All code should be written in **Python 3**.
This is the default in Google Colab.

In [1]:
!python --version

Python 3.10.19


If you want to run code on your own computer, then download this notebook through `File -> Download .ipynb`.
The easiest way to
install Python is through downloading
[Anaconda](https://www.anaconda.com/download).
After installation, you can start the notebook by typing `jupyter notebook filename.ipynb`.
You can also use an IDE
such as [PyCharm](https://www.jetbrains.com/pycharm/download/) to make
coding and debugging easier. It is good practice to create a [virtual
environment](https://docs.python.org/3/tutorial/venv.html) for this
project, so that any Python packages don’t interfere with other
projects.


**Learning Python 3**

If you are new to Python 3, you may want to check out a few of these resources:
- https://learnxinyminutes.com/docs/python3/
- https://www.learnpython.org/
- https://docs.python.org/3/tutorial/

In [2]:
import hashlib
import json
import math
import typing
from collections import Counter, defaultdict

import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import tqdm
from nltk.util import ngrams
from nltk.stem.porter import PorterStemmer
from scipy.sparse import csr_matrix

## Loading the data

**Download the sentiment lexicon and the movie reviews dataset.**

In [3]:
# download sentiment lexicon
!curl -L -o sent_lexicon https://gist.githubusercontent.com/bastings/d6f99dcb6c82231b94b013031356ba05/raw/f80a0281eba8621b122012c89c8b5e2200b39fd6/sent_lexicon
# download review data
!curl -L -o reviews.json https://gist.githubusercontent.com/bastings/d47423301cca214e3930061a5a75e177/raw/5113687382919e22b1f09ce71a8fecd1687a5760/reviews.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  647k  100  647k    0     0  1912k      0 --:--:-- --:--:-- --:--:-- 1925k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  6 79.6M    6 5201k    0     0  5075k      0  0:00:16  0:00:01  0:00:15 5089k
 14 79.6M   14 11.3M    0     0  5724k      0  0:00:14  0:00:02  0:00:12 5731k
 22 79.6M   22 17.5M    0     0  5940k      0  0:00:13  0:00:03  0:00:10 5946k
 31 79.6M   31 24.9M    0     0  6335k      0  0:00:12  0:00:04  0:00:08 6339k
 40 79.6M   40 32.2M    0     0  6555k      0  0:0

**Load the movie reviews.**

Each word in a review comes with its part-of-speech tag. For documentation on POS-tags, see https://catalog.ldc.upenn.edu/docs/LDC99T42/tagguid1.pdf.


In [4]:
# file structure:
# [
#  {"cv": integer, "sentiment": str, "content": list}
#  {"cv": integer, "sentiment": str, "content": list}
#   ..
# ]
# where `content` is a list of sentences,
# with a sentence being a list of (token, pos_tag) pairs.


with open("reviews.json", mode="r", encoding="utf-8") as f:
    reviews = json.load(f)

print("Total number of reviews:", len(reviews), "\n")


def print_sentence_with_pos(s):
    print(" ".join("%s/%s" % (token, pos_tag) for token, pos_tag in s))


for i, r in enumerate(reviews):
    print(r["cv"], r["sentiment"], len(r["content"]))  # cv, sentiment, num sents
    print_sentence_with_pos(r["content"][0])
    if i == 4:
        break

c = Counter()
for review in reviews:
    for sentence in review["content"]:
        for token, pos_tag in sentence:
            c[token.lower()] += 1

print("\nNumber of word types:", len(c))
print("Number of word tokens:", sum(c.values()))

print("\nMost common tokens:")
for token, count in c.most_common(20):
    print("%10s : %8d" % (token, count))

Total number of reviews: 2000 

0 NEG 29
Two/CD teen/JJ couples/NNS go/VBP to/TO a/DT church/NN party/NN ,/, drink/NN and/CC then/RB drive/NN ./.
1 NEG 11
Damn/JJ that/IN Y2K/CD bug/NN ./.
2 NEG 24
It/PRP is/VBZ movies/NNS like/IN these/DT that/WDT make/VBP a/DT jaded/JJ movie/NN viewer/NN thankful/JJ for/IN the/DT invention/NN of/IN the/DT Timex/NNP IndiGlo/NNP watch/NN ./.
3 NEG 19
QUEST/NN FOR/IN CAMELOT/NNP ``/`` Quest/NNP for/IN Camelot/NNP ''/'' is/VBZ Warner/NNP Bros./NNP '/POS first/JJ feature-length/JJ ,/, fully-animated/JJ attempt/NN to/TO steal/VB clout/NN from/IN Disney/NNP 's/POS cartoon/NN empire/NN ,/, but/CC the/DT mouse/NN has/VBZ no/DT reason/NN to/TO be/VB worried/VBN ./.
4 NEG 38
Synopsis/NNPS :/: A/DT mentally/RB unstable/JJ man/NN undergoing/VBG psychotherapy/NN saves/VBZ a/DT boy/NN from/IN a/DT potentially/RB fatal/JJ accident/NN and/CC then/RB falls/VBZ in/IN love/NN with/IN the/DT boy/NN 's/POS mother/NN ,/, a/DT fledgling/NN restauranteur/NN ./.

Number of wo

# Lexicon-based approach (4pts)



A traditional approach to automatically classify documents according to their sentiment is the lexicon-based approach. To implement this approach, you need a **sentiment lexicon**, i.e., a list of words annotated with a sentiment label (e.g., positive and negative) or a sentiment score (e.g., a score from 0 to 5).

In this practical, you will use the sentiment
lexicon released by Wilson et al. (2005). The path of the loaded lexicon is `"sent_lexicon"`.

> Theresa Wilson, Janyce Wiebe, and Paul Hoffmann
(2005). [Recognizing Contextual Polarity in Phrase-Level Sentiment
Analysis](http://www.aclweb.org/anthology/H/H05/H05-1044.pdf). HLT-EMNLP.

Pay attention to all the information available in the sentiment lexicon. The field *word1* contains the lemma, *priorpolarity* contains the sentiment label (positive, negative, both, or neutral), *type* gives you the magnitude of the word's sentiment (strong or weak), and *pos1* gives you the part-of-speech tag of the lemma. Some lemmas can have multiple part-of-speech tags and thus multiple entries in the lexicon. For those lemmas you are free to use any entry, so it is fine to only keep one, e.g. the first or last one.

In [5]:
with open("sent_lexicon", mode="r", encoding="utf-8") as f:
    line_cnt = 0
    for line in f:
        print(line.strip())
        line_cnt += 1
        if line_cnt > 4:
            break

type=weaksubj len=1 word1=abandoned pos1=adj stemmed1=n priorpolarity=negative
type=weaksubj len=1 word1=abandonment pos1=noun stemmed1=n priorpolarity=negative
type=weaksubj len=1 word1=abandon pos1=verb stemmed1=y priorpolarity=negative
type=strongsubj len=1 word1=abase pos1=verb stemmed1=y priorpolarity=negative
type=strongsubj len=1 word1=abasement pos1=anypos stemmed1=y priorpolarity=negative


Given such a sentiment lexicon, there are ways to solve
the classification task without using Machine Learning. For example, one might look up every word $w_1 ... w_n$ in a document, and compute a **binary score**
$S_{binary}$ by counting how many words have a positive or a
negative label in the sentiment lexicon $SLex$.

$$S_{binary}(w_1 w_2 ... w_n) = \sum_{i = 1}^{n}\text{sign}(SLex\big[w_i\big])$$

where $\text{sign}(SLex\big[w_i\big])$ refers to the polarity of $w_i$.

**Threshold.** On average, there are more positive than negative words per review (~7.13 more positive than negative per review) to take this bias into account you should use a threshold of **8** (roughly the bias itself) to make it harder to classify as positive.

$$
\text{classify}(S_{binary}(w_1 w_2 ... w_n)) = \bigg\{\begin{array}{ll}
        \text{positive} & \text{if } S_{binary}(w_1w_2...w_n) > threshold\\
        \text{negative} & \text{otherwise}
        \end{array}
$$


#### (Q1.1) Implement this classifier using the provided structure below. (1 pt)

In [ ]:
class SentimentLexicon:
    def __init__(
        self,
        lexicon_path: str = "./sent_lexicon",
        threshold: float = 8.0,
        weight: float = 1.0,
    ) -> None:
        """Sentiment classifier using a lexicon approach.

        Args:
            lexicon_path (str, optional): the location of the sentiment lexicon file. Defaults to "./sent_lexicon".
            threshold (float, optional): the threshold used in classification. Defaults to 8.0.
            weight (float, optional): the weight assigned to words with strong polarity. Defaults to 1.0.
        """
        self.lexicon_path = lexicon_path
        self.threshold = threshold
        self.weight = weight
        self.lexicon: dict[str, dict[str, str]] = {}

        # load and parse the sentiment lexicon file
        with open(self.lexicon_path, mode="r", encoding="utf-8") as f:
            for line in f:
                # each line contains key=value pairs separated by whitespace
                parts = line.strip().split()
                entry = {}

                # extract all key=value pairs
                for part in parts:
                    key, value = part.split("=")
                    entry[key] = value

                # if a lemma (word1) appears multiple times with different pos tags (pos1), keep only its last entry
                word = entry["word1"].lower()
                self.lexicon[word] = entry

    def fit(
        self, documents: list[list[str]], labels: list[typing.Literal["POS", "NEG"]]
    ) -> None:
        """Fit the classifer according to the input features and target labels.

        For the `SentimentLexicon` classifier, there are no parameters to learn.

        Args:
            documents (list[list[str]]): the list of documents as a list of tokens.
            labels (list[str]): the list of labels
        """
        pass # nothing to fit for the lexicon-based classifier

    def predict(self, documents: list[list[str]]) -> list[typing.Literal["POS", "NEG"]]:
        """Perform classification on input documents.

        Args:
            documents (list[list[str]]): the list of documents as a list of tokens.

        Returns:
            list[str]: predicted labels.
        """
        predictions = []

        for doc in documents:
            score = 0
            # compute sentiment score for the document
            for token in doc:
                token = token.lower()
                if token in self.lexicon:
                    polarity = self.lexicon[token].get("priorpolarity")
                    subj_type = self.lexicon[token].get("type")

                    # apply stronger weight for words with strong polarity
                    w = self.weight if subj_type == "strongsubj" else 1.0

                    # update sentiment score
                    if polarity == "positive":
                        score += w
                    elif polarity == "negative":
                        score -= w

                    # we ignore neutral words

            # classify as positive if score exceeds threshold
            label = "POS" if score >= self.threshold else "NEG"
            predictions.append(label)

        return predictions

#### (Q1.2) Implement functions to transform the dataset to the needed data structures. (0.5pt)

In our current implementation, the classifier's `fit` and `predict` methods expect a list of corpus documents represented as `list[list[str]]`, and labels as a `list[str]`. However, our data is represented as `list[dict[str, Any]]`. Before we can fit and classify, we'll need to define a function to transform our data to the desired data structure.

In [10]:
def extract_labels(
    documents: list[dict[str, typing.Any]],
) -> list[typing.Literal["POS", "NEG"]]:
    """Converts a list of reviews to a list of labels.

    Args:
        documents (list[dict[str, Any]]): the reviews as a list of dicts.

    Returns:
        list[str]: the labels as a list of str
    """
    # extract the value of the 'sentiment' key from each review
    labels = [doc["sentiment"] for doc in documents]
    return labels


def extract_unigrams(
    documents: list[dict[str, typing.Any]], lower: bool = True
) -> list[list[str]]:
    """Converts a list of reviews to a list of unigram token lists.

    Args:
        documents (list[dict[str, typing.Any]]): the reviews as a list of dicts.
        lower (bool, optional): whether to lowercase the review tokens. Defaults to True.

    Returns:
        list[list[str]]: the list of unigram tokens
    """
    # each review["content"] is a list of sentences,
    # where each sentence is a list of (token, pos) pairs
    unigrams = []

    for doc in documents:
        tokens = []
        for sentence in doc["content"]:
            for token, pos in sentence:
                tokens.append(token.lower() if lower else token)
        unigrams.append(tokens)
    return unigrams

#### (Q1.3) Implement a function to compute the accuracy of a classification. (0.5pt)

Finally, we need to implement a classification evaluation metric. For this practical, we'll be using [accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision#In_classification), which is defined as the proportion of correct classifications.

In [11]:
def accuracy(y_pred: list[str], y_true: list[str]) -> float:
    """Computes the accuracy score.

    Args:
        y_pred (list[str]): the predicted labels
        y_true (list[str]): the ground truth labels

    Returns:
        float: the accuracy score
    """
    # count how many predictions match the true labels
    correct = sum(p == t for p, t in zip(y_pred, y_true))
    # divide by total number of labels
    return correct / len(y_true) if y_true else 0.0

Now train, predict and evaluate your classifier using the implemented functions. Make sure to print the final accuracy score.

In [12]:
# prepare data
X_uni = extract_unigrams(reviews)
y = extract_labels(reviews)

print("First review unigram tokens:", X_uni[0])
print("First review label:", y[0])

First review unigram tokens: ['two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an', 'accident', '.', 'one', 'of', 'the', 'guys', 'dies', ',', 'but', 'his', 'girlfriend', 'continues', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and', 'has', 'nightmares', '.', 'what', "'s", 'the', 'deal', '?', 'watch', 'the', 'movie', 'and', '``', 'sorta', "''", 'find', 'out', '...', 'critique', ':', 'a', 'mind-fuck', 'movie', 'for', 'the', 'teen', 'generation', 'that', 'touches', 'on', 'a', 'very', 'cool', 'idea', ',', 'but', 'presents', 'it', 'in', 'a', 'very', 'bad', 'package', '.', 'which', 'is', 'what', 'makes', 'this', 'review', 'an', 'even', 'harder', 'one', 'to', 'write', ',', 'since', 'i', 'generally', 'applaud', 'films', 'which', 'attempt', 'to', 'break', 'the', 'mold', ',', 'mess', 'with', 'your', 'head', 'and', 'such', '-lrb-', 'lost', 'highway', '&', 'memento', '-rrb-', ',', 'but', 'there', 'are', 'good', 'and

In [9]:
# initialize the lexicon-based classifier with the given threshold and no weight
lexicon_clf = SentimentLexicon(lexicon_path="sent_lexicon", threshold=8.0, weight=1.0)

# no need to fit for the lexicon-based classifier
# lexicon_clf.fit(X_uni, y)

# predict sentiments
lexicon_clf_y_pred = lexicon_clf.predict(X_uni)

# evaluate accuracy
lexicon_clf_acc = accuracy(lexicon_clf_y_pred, y)

print(f"Test accuracy for the lexicon-based approach: {lexicon_clf_acc:.4f}")

Test accuracy for the lexicon-based approach: 0.6790


#### (Q1.4) Now incorporate magnitude information and report the classification accuracy. Don't forget to use the threshold. (1 pt)

As the sentiment lexicon also has information about the **magnitude** of
sentiment (e.g., *“excellent"* would have higher magnitude than
*“good"*), we can take a more fine-grained approach by adding up all
sentiment scores, and deciding the polarity of the movie review using
the sign of the weighted score $S_{weighted}$.

$$S_{weighted}(w_1w_2...w_n) = \sum_{i = 1}^{n}SLex\big[w_i\big]$$

In [13]:
# initialize the lexicon-based classifier with magnitude weighting
weighted_lexicon_clf = SentimentLexicon(lexicon_path="sent_lexicon", threshold=8.0, weight=2.0)

# no need to fit for the weighted lexicon-based classifier
# weighted_lexicon_clf.fit(X_uni, y)

# predict sentiments
weighted_lexicon_clf_y_pred = weighted_lexicon_clf.predict(X_uni)

# evaluate accuracy
weighted_lexicon_clf_acc = accuracy(weighted_lexicon_clf_y_pred, y)

print(f"Test accuracy for the weighted lexicon-based approach: {weighted_lexicon_clf_acc:.4f}")

Test accuracy for the weighted lexicon-based approach: 0.6830


> Let's test different weight values to see which one gives the highest accuracy.

In [45]:
# list of weights to try
weights = [0.1, 0.5, 1, 2.0, 3.0, 4.0,5.0, 10.0, 15.0, 25.0]

best_weight = None
best_accuracy = 0

for weight in weights:
    # initialize the lexicon-based classifier with the given weight
    lexicon_clf = SentimentLexicon(lexicon_path="sent_lexicon", threshold=8.0, weight=weight)

    # predict sentiments
    y_pred = lexicon_clf.predict(X_uni)

    # evaluate accuracy
    acc = accuracy(y_pred, y)
    print(f"Weight: {weight}, Accuracy: {acc:.4f}")

    # select best weight based on accuracy
    if acc > best_accuracy:
        best_accuracy = acc
        best_weight = weight

print(f"\nBest weight: {best_weight}, Best Accuracy: {best_accuracy:.4f}")

Weight: 0.1, Accuracy: 0.5780
Weight: 0.5, Accuracy: 0.6410
Weight: 1, Accuracy: 0.6790
Weight: 2.0, Accuracy: 0.6830
Weight: 3.0, Accuracy: 0.6855
Weight: 4.0, Accuracy: 0.6835
Weight: 5.0, Accuracy: 0.6840
Weight: 10.0, Accuracy: 0.6795
Weight: 15.0, Accuracy: 0.6770
Weight: 25.0, Accuracy: 0.6745

Best weight: 3.0, Best Accuracy: 0.6855


#### (Q1.5) A better threshold (1pt)

Above we have defined a threshold to account for an inherent bias in the dataset: there are more positive than negative words per review.
However, that threshold does not take into account *document length*. Explain why this is a problem and implement an alternative way to compute the threshold.

> The fixed threshold of 8 works fairly well, but it doesn’t take into account that longer reviews naturally contain more sentiment words. Because of that, they’re more likely to reach or exceed the threshold even if the overall tone is mixed, while shorter reviews might be marked as negative simply because they don’t have enough words to build up the score.
>
> To make this fairer, we normalize the sentiment score by the number of words that actually appear in the sentiment lexicon. This way, we’re looking at the **average sentiment per meaningful word** against a small bias that is used for dataset imbalance, and gives more balanced results for both short and long reviews.

Now implement your algorithm by adjusting the previously created `SentimentLexicon` class. Make sure to print the final accuracy score and compare to the previous results.

Note: you might need to adjust your threshold.

In [134]:
class BetterSentimentLexicon(SentimentLexicon):
    def predict(self, documents: list[list[str]]) -> list[str]:
        """Perform classification on input documents.

        Args:
            documents (list[list[str]]): the list of documents as a list of tokens.

        Returns:
            list[str]: predicted labels.
        """
        predictions = []
        bias = 0.1  # small bias to offset dataset imbalance

        for doc in documents:
            score = 0.0
            lexicon_tokens = 0  # count only words found in the lexicon

            # compute sentiment score for the document
            for token in doc:
                token = token.lower()
                if token in self.lexicon:
                    polarity = self.lexicon[token].get("priorpolarity")
                    subj_type = self.lexicon[token].get("type")

                    # apply stronger weight for words with strong polarity
                    w = self.weight if subj_type == "strongsubj" else 1.0

                    if polarity == "positive":
                        score += w
                    elif polarity == "negative":
                        score -= w

                    lexicon_tokens += 1

            # normalize only if there are lexicon words
            if lexicon_tokens > 0:
                norm_score = score / lexicon_tokens
            else:
                norm_score = 0.0

            # classify with bias threshold
            label = "POS" if norm_score > bias else "NEG"
            predictions.append(label)

        return predictions

In [135]:
# initialize the lexicon-based classifier no threshold and best weight
better_lexicon_clf = BetterSentimentLexicon(lexicon_path="sent_lexicon", weight=3.0)

# no need to fit for the better lexicon-based classifier
# better_lexicon_clf.fit(X_uni, y)

# predict sentiments
better_lexicon_clf_y_pred = better_lexicon_clf.predict(X_uni)

# evaluate accuracy
better_lexicon_clf_acc = accuracy(better_lexicon_clf_y_pred, y)

print(f"Test accuracy for the better lexicon-based approach: {better_lexicon_clf_acc:.4f}")

Test accuracy for the better lexicon-based approach: 0.6920


> We created a table that compares the performance of all lexicon-based sentiment classifiers tested so far. Each variation uses the same dataset and evaluation method, differing only in the use of weighting or threshold adjustments.
> 
> | Approach | Weight | Threshold | Accuracy |
> |:----------|:-------:|:-----------:|:----------:|
> | Basic Lexicon (Q1.3) | 1.0 | 8.0 | 0.6790 |
> | Weighted Lexicon (Q1.4) | 2.0 | 8.0 | 0.6830 |
> | Best Weight Search | 3.0 | 8.0 | 0.6855 |
> | Better Lexicon (Q1.5) | 3.0 | 0.1 (small bias) | **0.6920** |

# Naive Bayes (10pt)

Your second task is to program a simple Machine Learning approach that operates
on a simple Bag-of-Words (BoW) representation of the text data, as
described by Pang et al. (2002). In this approach, the only features we
will consider are the words in the text themselves, without bringing in
external sources of information. The BoW model is a popular way of
representing texts as vectors, making it
easy to apply classical Machine Learning algorithms on NLP tasks.
However, the BoW representation is also very crude, since it discards
all information related to word order and grammatical structure in the
original text—as the name suggests.

## Writing your own classifier (4pts)

Write your own code to implement the Naive Bayes (NB) classifier. As
a reminder, the Naive Bayes classifier works according to the following
equation:
$$\hat{c} = \operatorname*{arg\,max}_{c \in C} P(c|\bar{f}) = \operatorname*{arg\,max}_{c \in C} P(c)\prod^n_{i=1} P(f_i|c)$$
where $C = \{ \text{POS}, \text{NEG} \}$ is the set of possible classes,
$\hat{c} \in C$ is the most probable class, and $\bar{f}$ is the feature
vector. Remember that we use the log of these probabilities when making
a prediction:
$$\hat{c} = \operatorname*{arg\,max}_{c \in C} \Big\{\log P(c) + \sum^n_{i=1} \log P(f_i|c)\Big\}$$

You can find more details about Naive Bayes in [Jurafsky &
Martin](https://web.stanford.edu/~jurafsky/slp3/). You can also look at
this helpful
[pseudo-code](https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html).

*Note: this section and the next aim to put you in a position to replicate
    Pang et al.'s Naive Bayes results. However, your numerical results
    will differ from theirs, as they used different data.*

**You must write the Naive Bayes training and prediction code from
scratch.** You will not be given credit for using off-the-shelf Machine
Learning libraries.

The data contains the text of the reviews, where each document consists
of the sentences in the review, the sentiment of the review and an index
(cv) that you will later use for cross-validation. The
text has already been tokenised and POS-tagged for you. Your algorithm
should read in the text, **lowercase it**, store the words and their
frequencies in an appropriate data structure that allows for easy
computation of the probabilities used in the Naive Bayes algorithm, and
then make predictions for new instances.


#### (Q2.1) Unseen words (1pt)
The presence of words in the test dataset that
have not been seen during training can cause probabilities in the Naive Bayes classifier to equal $0$.
These can be words which are unseen in both positive and negative training reviews (case 1), but also words which are seen in reviews _of only one sentiment class_ in the training dataset (case 2). In both cases, **you should skip these words for both classes at test time**.  What would be the problem instead with skipping words only for one class in case 2?

> If we skip unseen words only for one class (for instance, when a word appears only in positive reviews during training but not in negative ones), we would **bias the prediction toward that class**.
> 
> This happens because the class that includes the word would still gain probability mass from it, while the other class would not, leading to an **unfair comparison of log-probabilities**.
> 
> By skipping the word for **both classes**, we ensure that both sides are treated equally and the decision is based only on words that were observed in **both sentiment classes**.

#### (Q2.2) Train your classifier on (positive and negative) reviews with cv-value 000-899, and test it on the remaining (positive and negative) reviews cv900–cv999.  Report results using classification accuracy as your evaluation metric. (2pts)


In [ ]:
class NaiveBayes:
    def __init__(self, smoothing: float = 0):
        """Naive Bayes classifier for multinomial data.

        Args:
            smoothing (float, optional): the Laplacian smoothing factor. Defaults to 0.
        """
        self.smoothing = smoothing

        # The datastructures necessary for NaiveBayes
        self.vocab: set[str] = set()

        self.prior: dict[typing.Literal["POS", "NEG"], float] = {
            "POS": 0,
            "NEG": 0,
        }

        self.token_counts: dict[str, dict[typing.Literal["POS", "NEG"], int]] = (
            defaultdict(
                lambda: {"POS": 0, "NEG": 0},
            )
        )

        self.cond_prob: dict[str, dict[typing.Literal["POS", "NEG"], float]] = (
            defaultdict(
                lambda: {"POS": 0.0, "NEG": 0.0},
            )
        )

    def fit(
        self,
        documents: list[list[typing.Any]],
        labels: list[typing.Literal["POS", "NEG"]],
    ) -> None:
        """Fit the classifer according to the input features and target labels.

        Args:
            documents (list[list[str]]): the list of documents as a list of tokens
            labels (list[str]): the list of labels
        """
        num_docs = len(documents)
        class_counts = Counter(labels)
        classes = ["POS", "NEG"]

        # compute prior probability P(c) for each class c
        for c in classes:
            self.prior[c] = class_counts[c] / num_docs

        # compute token counts per class
        for doc, label in zip(documents, labels):
            for token in doc:
                token_lower = token.lower()
                self.vocab.add(token_lower)
                self.token_counts[token_lower][label] += 1

        # compute total number of tokens per class
        total_tokens = {
            "POS": sum(self.token_counts[w]["POS"] for w in self.vocab),
            "NEG": sum(self.token_counts[w]["NEG"] for w in self.vocab),
        }

        V = len(self.vocab)
        alpha = self.smoothing

        # compute conditional probabilities P(word|class) with optional Laplacian smoothing
        for token in self.vocab:
            for c in classes:
                count_wc = self.token_counts[token][c]
                total_count_c = total_tokens[c]
                self.cond_prob[token][c] = (count_wc + alpha) / (total_count_c + alpha * V)

    def predict(
        self, documents: list[list[typing.Any]]
    ) -> list[typing.Literal["POS", "NEG"]]:
        """Perform classification on input documents.

        Args:
            documents (list[list[str]]): the list of documents as a list of tokens

        Returns:
            list[str]: predicted labels
        """
        predictions = []

        for doc in documents:
            scores = {}

            for c in ["POS", "NEG"]:

                # compute log-probability for class c for the test document
                if self.prior[c] == 0:
                    log_prob_c = float("-inf")
                else:
                    log_prob_c = math.log(self.prior[c])

                for token in doc:
                    token_lower = token.lower()
                    # skip unseen test words for both classes as discussed in (Q2.1)
                    pos_seen = self.token_counts[token_lower]["POS"] > 0
                    neg_seen = self.token_counts[token_lower]["NEG"] > 0

                    if not (pos_seen and neg_seen) and self.smoothing == 0.0:
                        continue
                    
                    # compute conditional log probabilities logP(word|class) 
                    prob_wc = self.cond_prob[token_lower][c]
                    if prob_wc > 0:
                        log_prob_c += math.log(prob_wc)
                    
                scores[c] = log_prob_c

            # assign the class with the higher log-probability
            predicted_class = max(scores, key=scores.get)
            predictions.append(predicted_class)

        return predictions

Now train, predict and evaluate your classifier on the reviews dataset. Make sure to select the right reviews for the train and test sets.

In [66]:
# split dataset based on cv index
train_reviews = [r for r in reviews if int(r["cv"]) < 900] # cv < 900 for training 
test_reviews = [r for r in reviews if int(r["cv"]) >= 900] # cv >= 900 for testing

# extract train data
X_train_nb = extract_unigrams(train_reviews)
y_train_nb = extract_labels(train_reviews)

# extract test data
X_test_nb = extract_unigrams(test_reviews)
y_test_nb = extract_labels(test_reviews)

In [67]:
# initialize the naive bayes classifier with no laplacian smoothing
nb_no_smooth = NaiveBayes(smoothing=0.0)

# fit the naive bayes classifier
nb_no_smooth.fit(X_train_nb, y_train_nb)

# predict sentiments
y_pred_nb_no_smooth = nb_no_smooth.predict(X_test_nb)

# evaluate accuracy
acc_nb_no_smooth = accuracy(y_pred_nb_no_smooth, y_test_nb)

print(f"Test accuracy for the Naive Bayes Classifier with no Laplacian Smoothing: {acc_nb_no_smooth:.4f}")

Test accuracy for the Naive Bayes Classifier with no Laplacian Smoothing: 0.8250


#### (Q2.3) Would you consider accuracy to also be a good way to evaluate your classifier in a situation where 90% of your data instances are of positive movie reviews? (1pt)

Simulate this scenario by keeping the positive reviews
data unchanged, but only using negative reviews cv000–cv089 for
training, and cv900–cv909 for testing. Calculate the classification
accuracy, and explain what changed.

> When 90% of the data is positive, **accuracy becomes misleading** because a classifier can achieve high accuracy simply by predicting the majority class ("POS") every time. This happens even if it completely fails to identify negative reviews. 
> 
> In such imbalanced scenarios, metrics like **precision, recall, or F1-score** are more informative, as they better reflect performance on the minority class.

In [68]:
train_reviews_imb = [
    r for r in reviews
    if (r["sentiment"] == "POS" and int(r["cv"]) < 900)  # keep all positive training reviews
    or (r["sentiment"] == "NEG" and int(r["cv"]) <= 89)   # only negatives cv000–cv089
]

test_reviews_imb = [
    r for r in reviews
    if (r["sentiment"] == "POS" and int(r["cv"]) >= 900)  # keep all positive test reviews
    or (r["sentiment"] == "NEG" and 900 <= int(r["cv"]) <= 909)  # only negatives cv900–cv909
]

# extract train data
X_train_nb_imb = extract_unigrams(train_reviews_imb)
y_train_nb_imb = extract_labels(train_reviews_imb)

# extract test data
X_test_nb_imb = extract_unigrams(test_reviews_imb)
y_test_nb_imb = extract_labels(test_reviews_imb)

In [72]:
# initialize the naive bayes classifier with no laplacian smoothing for imbalanced data
nb_imb = NaiveBayes(smoothing=0.0)

# fit the naive bayes classifier
nb_imb.fit(X_train_nb_imb, y_train_nb_imb)

# predict sentiments
y_pred_nb_imb = nb_imb.predict(X_test_nb_imb)

# evaluate accuracy
acc_nb_imb = accuracy(y_pred_nb_imb, y_test_nb_imb)

print(f"Test accuracy for the Naive Bayes Classifier with No Laplacian Smoothing for Imbalanced Data: {acc_nb_imb:.4f}")

Test accuracy for the Naive Bayes Classifier with No Laplacian Smoothing for Imbalanced Data: 0.6000


> When the dataset became imbalanced, the classifier’s accuracy dropped from about 0.82 to 0.60. This happened because the model was trained with far fewer negative examples and therefore learned to predict the positive class more often. As a result, accuracy decreased and became less meaningful as an evaluation metric, since it mainly reflects the dominant positive class rather than true performance across both classes.

## Smoothing (1pt)

As mentioned above, the presence of words in the test dataset that
have not been seen during training can cause probabilities in the Naive
Bayes classifier to be $0$, thus making that particular test instance
undecidable. The standard way to mitigate this effect (as well as to
give more clout to rare words) is to use smoothing, in which the
probability fraction

$$\frac{\text{count}(w_i, c)}{\sum\limits_{w\in V} \text{count}(w, c)}$$

for a word $w_i$ becomes

$$\frac{\text{count}(w_i, c) + \text{smoothing}(w_i)}{\sum\limits_{w\in V} \text{count}(w, c) + \sum\limits_{w \in V} \text{smoothing}(w)}$$


#### (Q2.4) Implement Laplace feature smoothing (1pt)
Implement Laplace smoothing, i.e., smoothing with a constant value ($smoothing(w) = \kappa, \forall w \in V$), in your Naive
Bayes classifier’s code, and report the accuracy for $\kappa = 1$ and $\kappa = 3.5$. Train your classifier on (positive and negative) reviews with cv-value 000-899, and test it on the remaining (positive and negative) reviews cv900–cv999.

In [73]:
# initialize the naive bayes classifier with κ=1 laplacian smoothing
nb_smooth_1 = NaiveBayes(smoothing=1.0)

# fit the naive bayes classifier
nb_smooth_1.fit(X_train_nb, y_train_nb)

# predict sentiments
y_pred_nb_smooth_1 = nb_smooth_1.predict(X_test_nb)

# evaluate accuracy
acc_nb_smooth_1 = accuracy(y_pred_nb_smooth_1, y_test_nb)

print(f"Test accuracy for the Naive Bayes Classifier with κ=1 Laplacian Smoothing: {acc_nb_smooth_1:.4f}")

Test accuracy for the Naive Bayes Classifier with κ=1 Laplacian Smoothing: 0.8250


In [74]:
# initialize the naive bayes classifier with κ=3.5 laplacian smoothing
nb_smooth_3_5 = NaiveBayes(smoothing=3.5)

# fit the naive bayes classifier
nb_smooth_3_5.fit(X_train_nb, y_train_nb)

# predict sentiments
y_pred_nb_smooth_3_5 = nb_smooth_3_5.predict(X_test_nb)

# evaluate accuracy
acc_nb_smooth_3_5 = accuracy(y_pred_nb_smooth_3_5, y_test_nb)

print(f"Test accuracy for the Naive Bayes Classifier with κ=3.5 Laplacian Smoothing: {acc_nb_smooth_3_5:.4f}")

Test accuracy for the Naive Bayes Classifier with κ=3.5 Laplacian Smoothing: 0.8450


**FROM NOW ON, ALWAYS USE SMOOTHING (YOU CAN CHOOSE $\kappa$) WHEN USING THE NAIVE BAYES CLASSIFIER**

## Cross-Validation (1pt)

A serious danger in using Machine Learning on small datasets, with many
iterations of slightly different versions of the algorithms, is ending up with Type III errors, also called the “testing hypotheses
suggested by the data” errors. This type of error occurs when we make
repeated improvements to our classifiers by playing with features and
their processing, but we don’t get a fresh, never-before seen test
dataset every time. Thus, we risk developing a classifier that gets better
and better on our data, but only gets worse at generalizing to new, unseen data. In other words, we risk developping a classifier that overfits.

A simple method to guard against Type III errors is to use
Cross-Validation. In **N-fold Cross-Validation**, we divide the data into N
distinct chunks, or folds. Then, we repeat the experiment N times: each
time holding out one of the folds for testing, training our classifier
on the remaining N - 1 data folds, and reporting performance on the
held-out fold. We can use different strategies for dividing the data:

-   Consecutive splitting:
  - cv000–cv099 = Split 1
  - cv100–cv199 = Split 2
  - etc.
  
-   Round-robin splitting (mod 10):
  - cv000, cv010, cv020, … = Split 1
  - cv001, cv011, cv021, … = Split 2
  - etc.

-   Random sampling/splitting
  - Not used here (but you may choose to split this way in a non-educational situation)



#### (Q2.5) Write the code to implement 10-fold cross-validation using round-robin splitting for your Naive Bayes classifier from Q2.4 and compute the 10 accuracies. Report the mean and standard deviation of the accuracies. (1pt)

In [76]:
def cross_validation_splits(
    documents: list[dict[str, typing.Any]],
    num_folds: int = 10,
) -> list[tuple[list, list]]:
    """Splits the reviews into disjoint subsets of train/test splits.

    Args:
        documents (list[dict[str, typing.Any]]): the reviews as dicts.
        num_folds (int, optional): the number of cross-validation folds. Defaults to 10.

    Returns:
        list[tuple[list, list]]: a list of train/test folds
    """
    folds = []

    # group reviews by their cv index modulo num_folds
    for i in range(num_folds):
        # test set: all reviews where cv number % num_folds == i
        test = [doc for doc in documents if int(doc["cv"]) % num_folds == i]
        # train set: all other reviews
        train = [doc for doc in documents if int(doc["cv"]) % num_folds != i]

        folds.append((train, test))

    return folds

Now re-train the Naive Bayes classifier on each fold, and evaluate on that fold's held-out test set. Make sure to print the accuracy mean and standard deviation.

Note: you may use `tqdm` to add progress bars

In [94]:
from tqdm import tqdm
from statistics import mean, stdev

folds = cross_validation_splits(reviews, num_folds=10)
nb_accuracies = []

for i, (train_docs, test_docs) in enumerate(tqdm(folds, desc="10-Fold Cross-Validation")):
    # extract train data
    X_train = extract_unigrams(train_docs)
    y_train = extract_labels(train_docs)

    # extract test data
    X_test = extract_unigrams(test_docs)
    y_test = extract_labels(test_docs)

    # initialize naive bayes classifier with κ=3.5 laplacian smoothing
    nb = NaiveBayes(smoothing=3.5)

    # fit the naive bayes classifier
    nb.fit(X_train, y_train)

    # predict sentiments
    y_pred = nb.predict(X_test)

    # evaluate accuracy
    acc = accuracy(y_pred, y_test)
    nb_accuracies.append(acc)

    tqdm.write(f"[Fold {i+1:02d}] Accuracy: {acc:.4f}")

# report overall performance
print("\n=== 10-Fold Cross-Validation Results (Naive Bayes, κ=3.5) ===")
print(f"Mean accuracy: {mean(nb_accuracies):.4f}")
print(f"Standard deviation: {stdev(nb_accuracies):.4f}")

10-Fold Cross-Validation:  10%|█         | 1/10 [00:11<01:45, 11.74s/it]

[Fold 01] Accuracy: 0.7900


10-Fold Cross-Validation:  20%|██        | 2/10 [00:18<01:11,  8.95s/it]

[Fold 02] Accuracy: 0.8450


10-Fold Cross-Validation:  30%|███       | 3/10 [00:20<00:40,  5.76s/it]

...

[Fold 08] Accuracy: 0.7950


10-Fold Cross-Validation:  90%|█████████ | 9/10 [00:31<00:02,  2.04s/it]

[Fold 09] Accuracy: 0.8450


10-Fold Cross-Validation: 100%|██████████| 10/10 [00:33<00:00,  3.31s/it]

[Fold 10] Accuracy: 0.8150

=== 10-Fold Cross-Validation Results (Naive Bayes, κ=3.5) ===
Mean accuracy: 0.8240
Standard deviation: 0.0246


**FROM NOW ON, ALWAYS USE CROSS VALIDATION WHEN EVALUATING A CLASSIFIER**

## Features, overfitting, and the curse of dimensionality

In the Bag-of-Words model, ideally we would like each distinct word in
the text to be mapped to its own dimension in the output vector
representation. However, real world text is messy, and we need to decide
on what we consider to be a word. For example, is “`word`" different
from “`Word`", from “`word`”, or from “`words`"? Too strict a
definition, and the number of features explodes, while our algorithm
fails to learn anything generalisable. Too lax, and we risk destroying
our learning signal. In the following section, you will learn about
confronting the feature sparsity and the overfitting problems as they
occur in NLP classification tasks.

### Stemming (1.5pts)

To make your algorithm more robust, use stemming and
hash different inflections of a word to the same feature in the BoW
vector space.


#### (Q2.6): How does the performance of your classifier change when you use stemming on your training and test datasets? (1pt)

First we'll need to adjust our `extract_unigrams` function to extract stemmed unigrams. Use the provided structure below.

In [78]:
def extract_stems(
    documents: list[dict[str, typing.Any]], stemmer: PorterStemmer, lower: bool = True
) -> list[list[str]]:
    """Extract stemmed unigrams from a corpus of documents.

    Args:
        documents (list[dict[str, Any]]): the list of documents
        stemmer (PorterStemmer): the stemmer to use when stemming documents
        lower (bool, optional): whether to lowercase tokens before stemming. Defaults to True.

    Returns:
        list[list[str]]: the list of token lists
    """
    # each review["content"] is a list of sentences,
    # where each sentence is a list of (token, pos) pairs
    stems = []

    for doc in documents:
        stemmed_tokens = []
        for sentence in doc["content"]:
            for token, pos in sentence:
                # lowercase if needed, then stem
                word = token.lower() if lower else token
                stemmed_word = stemmer.stem(word)
                stemmed_tokens.append(stemmed_word)
        stems.append(stemmed_tokens)

    return stems

In [79]:
# stemming example
stemmer = PorterStemmer()

X_stem = extract_stems(reviews, stemmer)
y = extract_labels(reviews)

print("First review stem tokens:", X_stem[0])
print("First review label:", y[0])

First review stem tokens: ['two', 'teen', 'coupl', 'go', 'to', 'a', 'church', 'parti', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an', 'accid', '.', 'one', 'of', 'the', 'guy', 'die', ',', 'but', 'hi', 'girlfriend', 'continu', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and', 'ha', 'nightmar', '.', 'what', "'s", 'the', 'deal', '?', 'watch', 'the', 'movi', 'and', '``', 'sorta', "''", 'find', 'out', '...', 'critiqu', ':', 'a', 'mind-fuck', 'movi', 'for', 'the', 'teen', 'gener', 'that', 'touch', 'on', 'a', 'veri', 'cool', 'idea', ',', 'but', 'present', 'it', 'in', 'a', 'veri', 'bad', 'packag', '.', 'which', 'is', 'what', 'make', 'thi', 'review', 'an', 'even', 'harder', 'one', 'to', 'write', ',', 'sinc', 'i', 'gener', 'applaud', 'film', 'which', 'attempt', 'to', 'break', 'the', 'mold', ',', 'mess', 'with', 'your', 'head', 'and', 'such', '-lrb-', 'lost', 'highway', '&', 'memento', '-rrb-', ',', 'but', 'there', 'are', 'good', 'and', 'bad', 'way', 'of', 'make', 'all'

Now re-train your Naive Bayes classifier. Use cross-validation to evaluate the classifier. Use nltk's `PorterStemmer` to stem the document tokens.


In [95]:
# initialize porter stemmer
stemmer = PorterStemmer()

stem_accuracies = []

for i, (train_docs, test_docs) in enumerate(tqdm(folds, desc="10-Fold Cross-Validation with Stemming")):
    # extract stemmed train data
    X_train = extract_stems(train_docs, stemmer)
    y_train = extract_labels(train_docs)

    # extract stemmed test data
    X_test = extract_stems(test_docs, stemmer)
    y_test = extract_labels(test_docs)

    # initialize naive bayes classifier with κ=3.5 laplacian smoothing
    nb = NaiveBayes(smoothing=3.5)

    # fit the naive bayes classifier
    nb.fit(X_train, y_train)

    # predict sentiments
    y_pred = nb.predict(X_test)

    # evaluate accuracy
    acc = accuracy(y_pred, y_test)
    stem_accuracies.append(acc)

    tqdm.write(f"[Fold {i+1:02d}] Accuracy: {acc:.4f}")

# report overall performance
print("\n=== 10-Fold Cross-Validation Results (Naive Bayes, κ=3.5, Stemming) ===")
print(f"Mean accuracy: {mean(stem_accuracies):.4f}")
print(f"Standard deviation: {stdev(stem_accuracies):.4f}")

10-Fold Cross-Validation with Stemming:  10%|█         | 1/10 [00:38<05:47, 38.63s/it]

[Fold 01] Accuracy: 0.7850


10-Fold Cross-Validation with Stemming:  20%|██        | 2/10 [01:03<04:04, 30.55s/it]

[Fold 02] Accuracy: 0.8400


10-Fold Cross-Validation with Stemming:  30%|███       | 3/10 [01:30<03:23, 29.04s/it]

...

[Fold 08] Accuracy: 0.7950


10-Fold Cross-Validation with Stemming:  90%|█████████ | 9/10 [04:32<00:30, 30.63s/it]

[Fold 09] Accuracy: 0.8300


10-Fold Cross-Validation with Stemming: 100%|██████████| 10/10 [05:29<00:00, 32.91s/it]


[Fold 10] Accuracy: 0.8250

=== 10-Fold Cross-Validation Results (Naive Bayes, κ=3.5, Stemming) ===
Mean accuracy: 0.8210
Standard deviation: 0.0237


#### (Q2.7) What happens to the number of features (i.e., the size of the vocabulary) when using stemming as opposed to (Q2.4)? (0.5pt)

Give actual numbers. You can use the training set from Q2.4 to determine these.

In [ ]:
# use the same training set from q2.4
X_train_unigrams = extract_unigrams(train_reviews) # this is the same as X_train_nb
X_train_stems = extract_stems(train_reviews, PorterStemmer())

# compute vocab sizes
vocab_unigrams = set(token for doc in X_train_unigrams for token in doc)
vocab_stems = set(token for doc in X_train_stems for token in doc)

print(f"Vocabulary Size without Stemming: {len(vocab_unigrams)}")
print(f"Vocabulary Size with Stemming: {len(vocab_stems)}")
print(f"Reduction: {len(vocab_unigrams) - len(vocab_stems)} fewer features ({100 * (1 - len(vocab_stems)/len(vocab_unigrams)):.2f}% reduction)")

Vocabulary Size without Stemming: 45348
Vocabulary Size with Stemming: 32404
Reduction: 12944 fewer features (28.54% reduction)


### N-grams (2.5pts)

A simple way of retaining some of the word
order information when using bag-of-words representations is to add **n-gram** features.






#### (Q2.8) Retrain your classifier from (Q2.4) using **unigrams+bigrams** and **unigrams+bigrams+trigrams** as features. (2pt)

We'll need to adjust our `extract_unigrams` function to extract n-grams. Use the provided structure below. You can use nltk's `ngrams` function to extract n-grams from the sentences.

In [87]:
def extract_ngrams(
    documents: list[dict], n_values: typing.Sequence[int], lower: bool = True
) -> list[list[typing.Any]]:
    """Extract n-gram features from the corpus of documents.

    Args:
        documents (list[dict]): the list of documents
        n_values (typing.Sequence[int]): the different sizes of n-grams to take (1=unigram, 2=bigram)
        lower (bool, optional): whether to lowercase the tokens. Defaults to True.

    Returns:
        list[list[tuple[str]]]: the n-gram tokens
    """
    ngrams_list = []

    for doc in documents:
        ngram_tokens = []

        # each review["content"] is a list of sentences,
        # where each sentence is a list of (token, pos) pairs
        for sentence in doc["content"]:
            tokens = [token.lower() if lower else token for token, pos in sentence]

            for n in n_values:
                for ng in ngrams(tokens, n):
                    # join tuple into a single string token
                    ngram_tokens.append(" ".join(ng))

        ngrams_list.append(ngram_tokens)

    return ngrams_list

In [89]:
# bigram example
X_bi = extract_ngrams(reviews, [2])
y = extract_labels(reviews)

print("First review bigram tokens:", X_bi[0])
print("First review label:", y[0])

First review bigram tokens: ['two teen', 'teen couples', 'couples go', 'go to', 'to a', 'a church', 'church party', 'party ,', ', drink', 'drink and', 'and then', 'then drive', 'drive .', 'they get', 'get into', 'into an', 'an accident', 'accident .', 'one of', 'of the', 'the guys', 'guys dies', 'dies ,', ', but', 'but his', 'his girlfriend', 'girlfriend continues', 'continues to', 'to see', 'see him', 'him in', 'in her', 'her life', 'life ,', ', and', 'and has', 'has nightmares', 'nightmares .', "what 's", "'s the", 'the deal', 'deal ?', 'watch the', 'the movie', 'movie and', 'and ``', '`` sorta', "sorta ''", "'' find", 'find out', 'out ...', '... critique', 'critique :', ': a', 'a mind-fuck', 'mind-fuck movie', 'movie for', 'for the', 'the teen', 'teen generation', 'generation that', 'that touches', 'touches on', 'on a', 'a very', 'very cool', 'cool idea', 'idea ,', ', but', 'but presents', 'presents it', 'it in', 'in a', 'a very', 'very bad', 'bad package', 'package .', 'which is', 

In [90]:
# trigram example
X_tri = extract_ngrams(reviews, [3])
y = extract_labels(reviews)

print("First review trigram tokens:", X_tri[0])
print("First review label:", y[0])

First review trigram tokens: ['two teen couples', 'teen couples go', 'couples go to', 'go to a', 'to a church', 'a church party', 'church party ,', 'party , drink', ', drink and', 'drink and then', 'and then drive', 'then drive .', 'they get into', 'get into an', 'into an accident', 'an accident .', 'one of the', 'of the guys', 'the guys dies', 'guys dies ,', 'dies , but', ', but his', 'but his girlfriend', 'his girlfriend continues', 'girlfriend continues to', 'continues to see', 'to see him', 'see him in', 'him in her', 'in her life', 'her life ,', 'life , and', ', and has', 'and has nightmares', 'has nightmares .', "what 's the", "'s the deal", 'the deal ?', 'watch the movie', 'the movie and', 'movie and ``', 'and `` sorta', "`` sorta ''", "sorta '' find", "'' find out", 'find out ...', 'out ... critique', '... critique :', 'critique : a', ': a mind-fuck', 'a mind-fuck movie', 'mind-fuck movie for', 'movie for the', 'for the teen', 'the teen generation', 'teen generation that', 'gen

In [91]:
# unigram + bigram + trigram example 
X_uni_bi_tri = extract_ngrams(reviews, [1, 2, 3])
y = extract_labels(reviews)

print("First review unigram + bigram + trigram tokens:", X_uni_bi_tri[0])
print("First review label:", y[0])

First review unigram + bigram + trigram tokens: ['two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'two teen', 'teen couples', 'couples go', 'go to', 'to a', 'a church', 'church party', 'party ,', ', drink', 'drink and', 'and then', 'then drive', 'drive .', 'two teen couples', 'teen couples go', 'couples go to', 'go to a', 'to a church', 'a church party', 'church party ,', 'party , drink', ', drink and', 'drink and then', 'and then drive', 'then drive .', 'they', 'get', 'into', 'an', 'accident', '.', 'they get', 'get into', 'into an', 'an accident', 'accident .', 'they get into', 'get into an', 'into an accident', 'an accident .', 'one', 'of', 'the', 'guys', 'dies', ',', 'but', 'his', 'girlfriend', 'continues', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and', 'has', 'nightmares', '.', 'one of', 'of the', 'the guys', 'guys dies', 'dies ,', ', but', 'but his', 'his girlfriend', 'girlfriend continues', 'continues to', 'to see', 's

Now re-train your Naive Bayes classifier with **uni- and bigram** tokens. Report accuracy and compare it with that of the approaches you have previously implemented. Use cross-validation when evaluating your classifier. (1pt)

In [96]:
uni_bi_accuracies = []

for i, (train_docs, test_docs) in enumerate(tqdm(folds, desc="10-Fold Cross-Validation with Uni- and Bigrams")):
    # extract unigram + bigram train data
    X_train = extract_ngrams(train_docs, [1, 2])
    y_train = extract_labels(train_docs)

    # extract unigram + bigram test data
    X_test = extract_ngrams(test_docs, [1, 2])
    y_test = extract_labels(test_docs)

    # initialize naive bayes classifier with κ=3.5 laplacian smoothing
    nb = NaiveBayes(smoothing=3.5)

    # fit the naive bayes classifier
    nb.fit(X_train, y_train)

    # predict sentiments
    y_pred = nb.predict(X_test)

    # evaluate accuracy
    acc = accuracy(y_pred, y_test)
    uni_bi_accuracies.append(acc)

    tqdm.write(f"[Fold {i+1:02d}] Accuracy: {acc:.4f}")

# report overall performance
print("\n=== Cross-Validation Results - Naive Bayes Uni- and Bigrams ===")
print(f"Mean accuracy: {mean(uni_bi_accuracies):.4f}")
print(f"Standard deviation: {stdev(uni_bi_accuracies):.4f}")

10-Fold Cross-Validation with Uni- and Bigrams:  10%|█         | 1/10 [00:15<02:20, 15.58s/it]

[Fold 01] Accuracy: 0.7450


10-Fold Cross-Validation with Uni- and Bigrams:  20%|██        | 2/10 [00:30<02:00, 15.10s/it]

[Fold 02] Accuracy: 0.7500


10-Fold Cross-Validation with Uni- and Bigrams:  30%|███       | 3/10 [00:38<01:24, 12.12s/it]

...

[Fold 08] Accuracy: 0.7650


10-Fold Cross-Validation with Uni- and Bigrams:  90%|█████████ | 9/10 [01:34<00:10, 10.01s/it]

[Fold 09] Accuracy: 0.7850


10-Fold Cross-Validation with Uni- and Bigrams: 100%|██████████| 10/10 [01:43<00:00, 10.38s/it]

[Fold 10] Accuracy: 0.7850

=== Cross-Validation Results - Naive Bayes Uni- and Bigrams ===
Mean accuracy: 0.7755
Standard deviation: 0.0186


Now re-train your Naive Bayes classifier with **uni-, bi- and trigram** tokens. Report accuracy and compare it with that of the approaches you have previously implemented. Use cross-validation when evaluating your classifier. (1pt)

In [136]:
uni_bi_tri_accuracies = []

for i, (train_docs, test_docs) in enumerate(tqdm(folds, desc="10-Fold Cross-Validation with Uni-, Bi- and Trigrams")):
    # extract unigram + bigram + trigram train data
    X_train = extract_ngrams(train_docs, [1, 2, 3])
    y_train = extract_labels(train_docs)

    # extract unigram + bigram + trigram test data
    X_test = extract_ngrams(test_docs, [1, 2, 3])
    y_test = extract_labels(test_docs)

    # initialize naive bayes classifier with κ=3.5 laplacian smoothing
    nb = NaiveBayes(smoothing=3.5)

    # fit the naive bayes classifier
    nb.fit(X_train, y_train)

    # predict sentiments
    y_pred = nb.predict(X_test)

    # evaluate accuracy
    acc = accuracy(y_pred, y_test)
    uni_bi_tri_accuracies.append(acc)

    tqdm.write(f"[Fold {i+1:02d}] Accuracy: {acc:.4f}")

# report overall performance
print("\n=== Cross-Validation Results - Naive Bayes Uni-, Bi- and Trigrams ===")
print(f"Mean accuracy: {mean(uni_bi_tri_accuracies):.4f}")
print(f"Standard deviation: {stdev(uni_bi_tri_accuracies):.4f}")

10-Fold Cross-Validation with Uni-, Bi- and Trigrams:  10%|█         | 1/10 [00:37<05:33, 37.11s/it]

[Fold 01] Accuracy: 0.7300


10-Fold Cross-Validation with Uni-, Bi- and Trigrams:  20%|██        | 2/10 [01:28<06:01, 45.22s/it]

[Fold 02] Accuracy: 0.7500


10-Fold Cross-Validation with Uni-, Bi- and Trigrams:  30%|███       | 3/10 [02:08<05:01, 43.13s/it]

...

[Fold 08] Accuracy: 0.7250


10-Fold Cross-Validation with Uni-, Bi- and Trigrams:  90%|█████████ | 9/10 [03:55<00:20, 20.49s/it]

[Fold 09] Accuracy: 0.7450


10-Fold Cross-Validation with Uni-, Bi- and Trigrams: 100%|██████████| 10/10 [04:14<00:00, 25.43s/it]

[Fold 10] Accuracy: 0.7650

=== Cross-Validation Results - Naive Bayes Uni-, Bi- and Trigrams ===
Mean accuracy: 0.7520
Standard deviation: 0.0170


#### (Q2.9): How many features does the BoW model have to take into account now? (0.5pt)
How would you expect the number of features to increase theoretically (e.g., linear, square, cubed, exponential)? How do the number of features increase in the held-out training set (compared to Q2.8)? Do you expect this rate of increase to continue for (much) larger n-grams?

Use the held-out training set from training set from Q2.4 for this.


> When we add bigrams and trigrams, the number of features increases a lot. In our results, there are about 45,000 unigrams, 465,000 unigrams + bigrams, and more than 1.3 million unigrams + bigrams + trigrams. This shows that the number of features grows much faster than linear, it increases almost exponentially because new word combinations are created for every n-gram. If we keep adding higher-order n-grams, the number of features would keep growing very quickly, making the model larger and more likely to overfit.

In [ ]:
# use the same training data from q2.4
X_train_uni = extract_ngrams(train_reviews, n_values=[1]) # only unigrams, same as X_train_nb and X_train_unigrams
X_train_uni_bi = extract_ngrams(train_reviews, n_values=[1, 2]) # unigrams + bigrams
X_train_uni_bi_tri = extract_ngrams(train_reviews, n_values=[1, 2, 3]) # unigrams + bigrams + trigrams

# compute vocab sizes
vocab_uni = set(token for doc in X_train_uni for token in doc)
vocab_uni_bi = set(token for doc in X_train_uni_bi for token in doc)
vocab_uni_bi_tri = set(token for doc in X_train_uni_bi_tri for token in doc)

print(f"Unigram Vocabulary Size: {len(vocab_uni)}")
print(f"Unigram + Bigram Vocabulary Size: {len(vocab_uni_bi)}")
print(f"Unigram + Bigram + Trigram Vocabulary Size: {len(vocab_uni_bi_tri)}")

print(f"Increase (Uni → Uni+Bi): {len(vocab_uni_bi) - len(vocab_uni)}")
print(f"Increase (Uni+Bi → Uni+Bi+Tri): {len(vocab_uni_bi_tri) - len(vocab_uni_bi)}")

Unigram Vocabulary Size: 45348
Unigram + Bigram Vocabulary Size: 465262
Unigram + Bigram + Trigram Vocabulary Size: 1346107
Increase (Uni → Uni+Bi): 419914
Increase (Uni+Bi → Uni+Bi+Tri): 880845




# (3) Support Vector Machines (4pts)

Though simple to understand, implement, and debug, one major problem with the Naive Bayes classifier is that its performance deteriorates (becomes skewed) when it is being used with features which are not independent (i.e., are correlated). Another popular classifier that doesn’t assume feature independence is the Support Vector Machine (SVM) classifier.

You can find more details about SVMs in Chapter 7 of Bishop: Pattern Recognition and Machine Learning.
Other sources for learning SVM:
* http://web.mit.edu/zoya/www/SVM.pdf
* http://www.cs.columbia.edu/~kathy/cs4701/documents/jason_svm_tutorial.pdf
* https://pythonprogramming.net/support-vector-machine-intro-machine-learning-tutorial/


#### (Q3.1): Train SVM and compare to Naive Bayes (2pts)

Train an SVM classifier (`sklearn.svm.LinearSVC`) using the unigram features collected for Naive Bayes.

sklearn's classifiers expect a different data structure than what we've been using so far. Instead of a list of tokens, we'll need to provide a (sparse) [document-term matrix](https://en.wikipedia.org/wiki/Document-term_matrix). Each row represents a document, and each column represents a token. The value in each cell represents how often a token appears in a document.

Define a function below that takes a list of tokens and constructs a document-term matrix. While not mandatory, it's recommended to use scipy's `csr_matrix` to produce a sparse matrix representation. This avoids having to keep many 0 values in memory, and can speed up SVM training.

Hint: the documentation on the [`csr_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html) is very helpful

In [99]:
def build_term_document_matrix(
    train_features: list[list[str]], test_features: list[list[str]]
) -> tuple[np.array, np.array]:
    """Converts a list of token lists to a document-term matrix.

    Args:
        train_features (list[list[str]]): the training token lists
        test_features (list[list[str]]): the testing token lists

    Returns:
        tuple[array, array]: a tuple of training and testing DTMs
    """
    # build the vocabulary from training data
    vocab = {}
    for doc in train_features:
        for token in doc:
            if token not in vocab:
                vocab[token] = len(vocab)

    vocab_size = len(vocab)

    # helper function to convert documents to csr sparse matrix, otherwise we would have a lot of repeated code
    def docs_to_csr(docs: list[list[str]]) -> csr_matrix:
        rows, cols, data = [], [], []

        for row_idx, doc in enumerate(docs):
            token_counts = Counter(doc)
            for token, count in token_counts.items():
                if token in vocab: 
                    col_idx = vocab[token]
                    rows.append(row_idx)
                    cols.append(col_idx)
                    data.append(count)

        return csr_matrix((data, (rows, cols)), shape=(len(docs), vocab_size), dtype=np.float64)

    X_train = docs_to_csr(train_features)
    X_test = docs_to_csr(test_features)

    return X_train, X_test

Besides a document term matrix, sklearn also expects the labels to be a list of `int`. Using the structure provide, implement a function to convert the `str` labels to integers.

In [100]:
def convert_labels_to_ints(labels: list[str]) -> list[int]:
    """Converts a list of "POS" or "NEG" to 0 or 1.

    Args:
        labels (list[str]): the list of str labels

    Returns:
        list[int]: the list of int labels
    """
    label_map = {"POS": 1, "NEG": 0}
    return [label_map[label] for label in labels]

Now train, predict and evaluate an SVM classifier. Compare the classification performance of the SVM classifier to that of the Naive Bayes classifier with smoothing. Use cross-validation to evaluate the performance of the classifiers.

You are not required to perform hyperparameter tuning, but it might be helpful.

In [102]:
from tqdm import tqdm
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from statistics import mean, stdev

svm_accuracies = []
nb_accuracies = []

# 10-fold cross-validation
for i, (train_docs, test_docs) in enumerate(tqdm(folds, desc="10-Fold Cross-Validation (SVM vs NB)")):
    # extract features
    X_train_features = extract_unigrams(train_docs)
    X_test_features = extract_unigrams(test_docs)

    # prepare data for naive bayes
    X_train_nb = X_train_features
    X_test_nb = X_test_features

    # convert to document-term matrices for svm
    X_train_svm, X_test_svm = build_term_document_matrix(X_train_features, X_test_features)

    # prepare labels for naive bayes
    y_train_nb = extract_labels(train_docs)
    y_test_nb = extract_labels(test_docs)

    # convert labels to ints for svm
    y_train_svm = convert_labels_to_ints(y_train)
    y_test_svm = convert_labels_to_ints(y_test)

    # train and evaluate svm
    svm = LinearSVC(random_state=42, max_iter=5000)
    svm.fit(X_train_svm, y_train_svm)
    y_pred_svm = svm.predict(X_test_svm)
    acc_svm = accuracy_score(y_test_svm, y_pred_svm)
    svm_accuracies.append(acc_svm)

    # train and evaluate naive bayes with κ=3.5 laplace smoothing
    nb = NaiveBayes(smoothing=3.5)
    nb.fit(X_train_nb, y_train_nb)
    y_pred_nb = nb.predict(X_test_nb)
    acc_nb = accuracy(y_pred_nb, y_test_nb)
    nb_accuracies.append(acc_nb)

    tqdm.write(f"[Fold {i+1:02d}] SVM Acc={acc_svm:.4f} | NB Acc={acc_nb:.4f}")

# report overall results
print("\n=== Cross-Validation Results (SVM vs NB) ===")
print(f"SVM Mean Accuracy: {mean(svm_accuracies):.4f}  (std: {stdev(svm_accuracies):.4f})")
print(f"NB Mean Accuracy: {mean(nb_accuracies):.4f}  (std: {stdev(nb_accuracies):.4f})")

10-Fold Cross-Validation (SVM vs NB):  10%|█         | 1/10 [00:17<02:37, 17.49s/it]

[Fold 01] SVM Acc=0.8100 | NB Acc=0.7900


10-Fold Cross-Validation (SVM vs NB):  20%|██        | 2/10 [00:30<01:58, 14.82s/it]

[Fold 02] SVM Acc=0.7950 | NB Acc=0.8450


10-Fold Cross-Validation (SVM vs NB):  30%|███       | 3/10 [00:47<01:50, 15.82s/it]

...

[Fold 08] SVM Acc=0.8500 | NB Acc=0.7950


10-Fold Cross-Validation (SVM vs NB):  90%|█████████ | 9/10 [01:50<00:11, 11.80s/it]

[Fold 09] SVM Acc=0.8750 | NB Acc=0.8450


10-Fold Cross-Validation (SVM vs NB): 100%|██████████| 10/10 [01:55<00:00, 11.59s/it]

[Fold 10] SVM Acc=0.8400 | NB Acc=0.8150

=== Cross-Validation Results (SVM vs NB) ===
SVM Mean Accuracy: 0.8320  (std: 0.0257)
NB Mean Accuracy: 0.8240  (std: 0.0246)


### POS disambiguation (2pts)

Now add in part-of-speech features. You will find the
movie review dataset has already been POS-tagged for you ([here](https://catalog.ldc.upenn.edu/docs/LDC99T42/tagguid1.pdf) you find the tagset). Try to
replicate the results obtained by Pang et al. (2002).



#### (Q3.2) Replace your features with word+POS features, and report performance with the SVM. Use cross-validation to evaluate the classifier and compare the results with (Q3.1). Does part-of-speech information help? Explain why this may be the case. (1pt)

> When using **word + POS features**, the accuracy increased slightly compared to using only unigrams. This shows that part-of-speech information helps because it adds extra context about how each word is used in a sentence. For example, the word **“love”** as a `verb` (“I love this movie”) expresses strong positive sentiment, while as a `noun` in a phrase like “a love of money” can carry a more negative tone.

Once again, we'll need to adjust our `extract_unigrams` function to extract (token, pos) tuples. Use the provided structure below.

In [103]:
def extract_pos_unigrams(
    documents: list[dict], lower: bool = True
) -> list[list[tuple[str, str]]]:
    """Extracts (token, pos) tuples.

    Args:
        documents (list[dict]): the list of documents
        lower (bool, optional): whether to lowercase words. Defaults to True.

    Returns:
        list[list[tuple[str, str]]]: the list of (token, pos) tuple lists
    """
    pos_unigrams = []

    for doc in documents:
        pos_tokens = []

        # each review["content"] is a list of sentences,
        # where each sentence is a list of (token, pos) pairs
        for sentence in doc["content"]:
            for token, pos in sentence:
                word = token.lower() if lower else token
                # combine the word and POS tag into a single feature
                combined = f"{word}_{pos}"
                pos_tokens.append(combined)

        pos_unigrams.append(pos_tokens)

    return pos_unigrams

In [ ]:
# pos unigrams example
X_pos_uni = extract_pos_unigrams(reviews)
y = extract_labels(reviews)

print("First review POS unigram tokens:", X_pos_uni[0])
print("First review label:", y[0])

First review POS unigram tokens: ['two_CD', 'teen_JJ', 'couples_NNS', 'go_VBP', 'to_TO', 'a_DT', 'church_NN', 'party_NN', ',_,', 'drink_NN', 'and_CC', 'then_RB', 'drive_NN', '._.', 'they_PRP', 'get_VBP', 'into_IN', 'an_DT', 'accident_NN', '._.', 'one_CD', 'of_IN', 'the_DT', 'guys_NNS', 'dies_VBZ', ',_,', 'but_CC', 'his_PRP$', 'girlfriend_NN', 'continues_VBZ', 'to_TO', 'see_VB', 'him_PRP', 'in_IN', 'her_PRP$', 'life_NN', ',_,', 'and_CC', 'has_VBZ', 'nightmares_NNS', '._.', 'what_WP', "'s_VBZ", 'the_DT', 'deal_NN', '?_.', 'watch_VB', 'the_DT', 'movie_NN', 'and_CC', '``_``', 'sorta_NN', "''_''", 'find_VB', 'out_RP', '..._:', 'critique_NNP', ':_:', 'a_NNP', 'mind-fuck_JJ', 'movie_NN', 'for_IN', 'the_DT', 'teen_NN', 'generation_NN', 'that_WDT', 'touches_NNS', 'on_IN', 'a_DT', 'very_RB', 'cool_JJ', 'idea_NN', ',_,', 'but_CC', 'presents_VBZ', 'it_PRP', 'in_IN', 'a_DT', 'very_RB', 'bad_JJ', 'package_NN', '._.', 'which_WDT', 'is_VBZ', 'what_WP', 'makes_VBZ', 'this_DT', 'review_NN', 'an_DT', 'ev

Now train, predict and evaluate your SVM classifier, and compare to the SVM with only unigram features.

In [108]:
pos_uni_accuracies = []

for i, (train_docs, test_docs) in enumerate(tqdm(folds, desc="10-Fold Cross-Validation with SVM POS Tag Unigrams")):
    # extract pos unigram train data
    X_train_features = extract_pos_unigrams(train_docs)
    y_train = extract_labels(train_docs)

    # extract pos unigram test data
    X_test_features = extract_pos_unigrams(test_docs)
    y_test = extract_labels(test_docs)

     # convert to document-term matrices for svm
    X_train, X_test = build_term_document_matrix(X_train_features, X_test_features)

    # convert labels to ints for svm
    y_train = convert_labels_to_ints(y_train)
    y_test = convert_labels_to_ints(y_test)

    # initialize svm classifier
    svm = LinearSVC(random_state=42, max_iter=8000)

    # fit the svm classifier
    svm.fit(X_train, y_train)

    # predict sentiments
    y_pred = svm.predict(X_test)

    # evaluate accuracy
    acc = accuracy(y_pred, y_test)
    pos_uni_accuracies.append(acc)

    tqdm.write(f"[Fold {i+1:02d}] Accuracy: {acc:.4f}")

# report overall performance
print("\n=== Cross-Validation Results - SVM POS Tag Unigrams ===")
print(f"Mean accuracy: {mean(pos_uni_accuracies):.4f}")
print(f"Standard deviation: {stdev(pos_uni_accuracies):.4f}")

10-Fold Cross-Validation with SVM POS Tag Unigrams:   0%|          | 0/10 [00:00<?, ?it/s]

10-Fold Cross-Validation with SVM POS Tag Unigrams:  10%|█         | 1/10 [00:02<00:24,  2.72s/it]

[Fold 01] Accuracy: 0.8200


10-Fold Cross-Validation with SVM POS Tag Unigrams:  20%|██        | 2/10 [00:05<00:23,  2.93s/it]

[Fold 02] Accuracy: 0.7950


...

[Fold 08] Accuracy: 0.8550


10-Fold Cross-Validation with SVM POS Tag Unigrams:  90%|█████████ | 9/10 [01:33<00:11, 11.51s/it]

[Fold 09] Accuracy: 0.8650


10-Fold Cross-Validation with SVM POS Tag Unigrams: 100%|██████████| 10/10 [01:36<00:00,  9.63s/it]

[Fold 10] Accuracy: 0.8400

=== Cross-Validation Results - SVM POS Tag Unigrams ===
Mean accuracy: 0.8380
Standard deviation: 0.0203


#### (Q3.3) Discard all closed-class words from your data (keep only nouns, verbs, adjectives, and adverbs), and report performance. Does this help? Use cross-validation to evaluate the classifier and compare the results with (Q3.2). Are closed-class words detrimental to the classifier? Explain why this may be the case. (1pt)

> When keeping only **open-class words** (nouns, verbs, adjectives, and adverbs), the accuracy increased slightly compared to using all POS-tagged words. This suggests that closed-class words (such as prepositions, determiners, and conjunctions) add little to sentiment analysis and can even introduce noise. Open-class words carry most of the emotional and descriptive meaning in a sentence, while closed-class words mainly serve grammatical functions and rarely reflect sentiment. So by removing them, the classifier focuses on more informative features, leading to better performance.

For the final time, we'll need to adjust our `extract_pos_unigrams` function to filter (token, pos) tuples for open-class words. Use the provided structure below.

In [109]:
def extract_open_class_pos_unigrams(
    documents: list[dict], lower: bool = True
) -> list[list[tuple[str, str]]]:
    """Extracts (token, pos) tuples for open-class words only.

    Args:
        documents (list[dict]): the list of documents
        lower (bool, optional): whether to lowercase words. Defaults to True.

    Returns:
        list[list[tuple[str, str]]]: the list of (token, pos) tuple lists
    """
    # penn treebank open-class pos tags
    open_class_tags = {"NN", "NNS", "NNP", "NNPS",  # nouns
                       "VB", "VBD", "VBG", "VBN", "VBP", "VBZ",  # verbs
                       "JJ", "JJR", "JJS",  # adjectives
                       "RB", "RBR", "RBS"}  # adverbs

    open_class_pos_unigrams = []

    for doc in documents:
        pos_tokens = []
        
        # each review["content"] is a list of sentences,
        # where each sentence is a list of (token, pos) pairs
        for sentence in doc["content"]:
            for token, pos in sentence:
                if pos in open_class_tags:
                    # combine the word and open class pos tag into a single feature
                    word = token.lower() if lower else token
                    combined = f"{word}_{pos}"
                    pos_tokens.append(combined)

        open_class_pos_unigrams.append(pos_tokens)

    return open_class_pos_unigrams

In [110]:
# open class pos unigrams example
X_open_class_pos_uni = extract_open_class_pos_unigrams(reviews)
y = extract_labels(reviews)

print("First review open class POS unigram tokens:", X_open_class_pos_uni[0])
print("First review label:", y[0])

First review open class POS unigram tokens: ['teen_JJ', 'couples_NNS', 'go_VBP', 'church_NN', 'party_NN', 'drink_NN', 'then_RB', 'drive_NN', 'get_VBP', 'accident_NN', 'guys_NNS', 'dies_VBZ', 'girlfriend_NN', 'continues_VBZ', 'see_VB', 'life_NN', 'has_VBZ', 'nightmares_NNS', "'s_VBZ", 'deal_NN', 'watch_VB', 'movie_NN', 'sorta_NN', 'find_VB', 'critique_NNP', 'a_NNP', 'mind-fuck_JJ', 'movie_NN', 'teen_NN', 'generation_NN', 'touches_NNS', 'very_RB', 'cool_JJ', 'idea_NN', 'presents_VBZ', 'very_RB', 'bad_JJ', 'package_NN', 'is_VBZ', 'makes_VBZ', 'review_NN', 'even_RB', 'harder_RBR', 'write_VB', 'generally_RB', 'applaud_VBP', 'films_NNS', 'attempt_VBP', 'break_VB', 'mold_NN', 'mess_NN', 'head_NN', 'such_JJ', 'lost_JJ', 'highway_NNP', 'memento_NNP', 'are_VBP', 'good_JJ', 'bad_JJ', 'ways_NNS', 'making_VBG', 'types_NNS', 'films_NNS', 'folks_NNS', 'just_RB', 'did_VBD', "n't_RB", 'snag_NN', 'correctly_RB', 'seem_VBP', 'have_VB', 'taken_VBN', 'pretty_RB', 'neat_JJ', 'concept_NN', 'executed_VBD', 't

Now train, predict and evaluate your SVM classifier, and compare to the SVM with all POS tokens.

In [112]:
open_class_pos_uni_accuracies = []

for i, (train_docs, test_docs) in enumerate(tqdm(folds, desc="10-Fold Cross-Validation with SVM Open Class POS Tag Unigrams")):
    # extract open class pos unigram train data
    X_train_features = extract_open_class_pos_unigrams(train_docs)
    y_train = extract_labels(train_docs)

    # extract open class pos unigram test data
    X_test_features = extract_open_class_pos_unigrams(test_docs)
    y_test = extract_labels(test_docs)

     # convert to document-term matrices for svm
    X_train, X_test = build_term_document_matrix(X_train_features, X_test_features)

    # convert labels to ints for svm
    y_train = convert_labels_to_ints(y_train)
    y_test = convert_labels_to_ints(y_test)

    # initialize svm classifier
    svm = LinearSVC(random_state=42, max_iter=8000)

    # fit the svm classifier
    svm.fit(X_train, y_train)

    # predict sentiments
    y_pred = svm.predict(X_test)

    # evaluate accuracy
    acc = accuracy(y_pred, y_test)
    open_class_pos_uni_accuracies.append(acc)

    tqdm.write(f"[Fold {i+1:02d}] Accuracy: {acc:.4f}")

# report overall performance
print("\n=== Cross-Validation Results - SVM Open Class POS Tag Unigrams ===")
print(f"Mean accuracy: {mean(open_class_pos_uni_accuracies):.4f}")
print(f"Standard deviation: {stdev(open_class_pos_uni_accuracies):.4f}")

10-Fold Cross-Validation with SVM Open Class POS Tag Unigrams:  10%|█         | 1/10 [00:04<00:36,  4.07s/it]

[Fold 01] Accuracy: 0.8250


10-Fold Cross-Validation with SVM Open Class POS Tag Unigrams:  20%|██        | 2/10 [00:10<00:42,  5.28s/it]

[Fold 02] Accuracy: 0.8350


10-Fold Cross-Validation with SVM Open Class POS Tag Unigrams:  30%|███       | 3/10 [00:12<00:28,  4.00s/it]

...

[Fold 08] Accuracy: 0.8600


10-Fold Cross-Validation with SVM Open Class POS Tag Unigrams:  90%|█████████ | 9/10 [00:27<00:02,  2.50s/it]

[Fold 09] Accuracy: 0.8750


10-Fold Cross-Validation with SVM Open Class POS Tag Unigrams: 100%|██████████| 10/10 [00:28<00:00,  2.82s/it]

[Fold 10] Accuracy: 0.8350

=== Cross-Validation Results - SVM Open Class POS Tag Unigrams ===
Mean accuracy: 0.8490
Standard deviation: 0.0200


# (4) Discussion (max. 500 words). (5pts)

> Based on your experiments, what are the effective features and techniques in sentiment analysis? What information do different features encode?
Why is this important? What are the limitations of these features and techniques?

> Our experiments gave us a clear view of what makes features and techniques effective in sentiment analysis. We started with **lexicon-based methods**, which rely on predefined word lists to decide if a text is positive or negative. These methods achieved modest accuracy (around **0.68**) and gave us a simple but useful baseline. Adding weights to strong sentiment words and normalizing by review length to reduce bias slightly improved results to **0.69**, showing that small adjustments can make rule-based models fairer for reviews of different sizes. However, the biggest weakness of such methods is the lack of context understanding. Phrases like “not bad” or “too good to be true” would often be misclassified because the system can only look at individual word polarity. Lexicon-based methods also fail to capture subtle emotional tone or mixed feelings, which are common in real reviews.
>
> When we moved to **Naive Bayes**, accuracy improved sharply to **0.83–0.85**, showing the power of data-driven learning. The model could estimate how likely each word was to appear in positive or negative reviews, which made it far more adaptable than fixed lexicons. Adding **Laplacian smoothing (κ=3.5)** helped handle unseen words and achieved the best performance. **Stemming** reduced the vocabulary size and made the model slightly more general, though it also lost some nuance by grouping words with different meanings together. When we added **bigrams and trigrams**, the feature space exploded (from about 45k to over 1.3 million features), leading to overfitting and a drop in accuracy to around **0.75**. This reminded us that more features do not always mean better performance. In general, Naive Bayes performed well because of its simplicity and speed, but it struggled with word dependencies and contextual meaning.
>
> The **SVM classifier** achieved the best overall accuracy at **0.849**, especially when we used **open-class POS features** (nouns, verbs, adjectives, and adverbs). SVMs handle correlated features much better than Naive Bayes and can find more precise decision boundaries between sentiment classes. Adding **POS tags** to the simple SVM gave a small but consistent boost, as it helped the model distinguish between how words were used in context. For instance, “love” as a verb (“I love it”) is clearly positive, while as a noun (“a love of power”) can carry a negative tone. Removing closed-class words like prepositions and conjunctions helped reduce noise and focus on emotionally meaningful terms.
>
> In summary, our experiments showed that combining simple representations with a bit of linguistic structure gives the best performance. The lexicon-based methods were easy to interpret but limited by their lack of context. Naive Bayes improved accuracy greatly by learning from word distributions, especially with Laplacian smoothing, while the SVM with open-class POS features achieved the best results. Overall, careful feature selection, such as focusing on informative words and using part-of-speech cues, proved more important than model complexity for achieving strong, reliable sentiment classification.
>
> | Model | Approach | 10-Fold Cross Validation | Mean Test Accuracy |
> |:------|:----------|:----------------:|:------------------:|
> | **Lexicon-Based** | Threshold=0.8 & Weight=1.0 | No | 0.6790 |
> | **Lexicon-Based** | Threshold=0.8 & Best Weight=3.0 | No | 0.6855 |
> | **Lexicon-Based** | Normalized Threshold & Best Weight=3.0 | No | 0.6920 |
> | **Naive Bayes** | No Laplacian Smoothing | No | 0.8250 |
> | **Naive Bayes** | Laplacian Smoothing (κ=1.0) | No | 0.8250 |
> | **Naive Bayes** | Laplacian Smoothing (κ=3.5) | No | **0.8450** |
> | **Naive Bayes** | Laplacian Smoothing (κ=3.5) & Stemming | Yes | 0.8210 ± 0.0237 |
> | **Naive Bayes** | Laplacian Smoothing (κ=3.5) & Uni- + Bigrams | Yes | 0.7755 ± 0.0186 |
> | **Naive Bayes** | Laplacian Smoothing (κ=3.5) & Uni- + Bi- + Trigrams | Yes | 0.7520 ± 0.0170 |
> | **SVM** | Unigrams | Yes | 0.8320 ± 0.0257 |
> | **SVM** | Unigrams & POS Tags | Yes | 0.8380 ± 0.0203 |
> | **SVM** | Unigrams & Open-Class POS Tags | Yes | **0.8490 ± 0.0200** |

*Write your answer here in up to 500 words (-0.25pt for >50 extra words, -0.5 points for >100 extra words, ...)*.


# Use of AI tools

By submitting this notebook for grading you testify that:
* AI did not draft an earlier version of your work.
* You did not use AI-powered code completion.
* You did not implement algorithms suggested by an AI tool.
* AI did not revise a version of your work.
* You did not implement suggestions made by an AI tool.

You in the sentences above refers to you and your team member(s). AI refers to LM-based tools and assistants (e.g. ChatGPT, UvA AI Chat, Gemini, etc.).

If you did make use of an AI tool, you should describe the uses you made of it below. Or indicate that no such tool was used.

> No such tool was used.